<a href="https://colab.research.google.com/github/aritanaoya/keibaAI/blob/main/scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# スクレイピング

### 以下4種類のデータをスクレイピングする。

- レース結果データ	Results.scrape()	results.pickle
- 馬の過去成績データ	HorseResults.scrape()	horse_results.pickle
- 血統データ	Peds.scrape()	peds.pickle

## モジュールのインポート

In [1]:
import pandas as pd
import numpy as np
import datetime
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
import re
from urllib.request import urlopen

## Results.scrape()

メインとなる訓練データである、レース結果データをスクレイピングするメソッド

In [2]:
class Results:
    @staticmethod
    def scrape(race_id_list):
        """
        レース結果データをスクレイピングする関数

        Parameters:
        ----------
        race_id_list : list
            レースIDのリスト

        Returns:
        ----------
        race_results_df : pandas.DataFrame
            全レース結果データをまとめてDataFrame型にしたもの
        """

        #race_idをkeyにしてDataFrame型を格納
        race_results = {}
        for race_id in tqdm(race_id_list):
          time.sleep(1)
          try:
            url = "https://db.netkeiba.com/race/" + race_id
            #メインとなるテーブルデータを取得
            df = pd.read_html(url)[0]

            html = requests.get(url)
            html.encoding = "EUC-JP"
            soup = BeautifulSoup(html.text, "html.parser")

            #天候、レースの種類、コースの長さ、馬場の状態、日付をスクレイピング
            #NetKeibaの例     2歳未勝利
            #                 芝右1800m / 天候 : 曇 / 芝 : 良 / 発走 : 09:50
            texts = (
                soup.find("div", attrs={"class": "data_intro"}).find_all("p")[0].text
                + soup.find("div", attrs={"class": "data_intro"}).find_all("p")[1].text
            )
            info = re.findall(r'\w+', texts)
            for text in info:
                if text in ["芝", "ダート"]:
                    df["race_type"] = [text] * len(df)
                if "障" in text:
                    df["race_type"] = ["障害"] * len(df)
                if "m" in text:
                    df["course_len"] = [int(re.findall(r"\d+", text)[0])] * len(df)
                if text in ["良", "稍重", "重", "不良"]:
                    df["ground_state"] = [text] * len(df)
                if text in ["曇", "晴", "雨", "小雨", "小雪", "雪"]:
                    df["weather"] = [text] * len(df)
                if "年" in text:
                    df["date"] = [text] * len(df)

            #馬ID、騎手IDをスクレイピング
            horse_id_list = []
            horse_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                "a", attrs={"href": re.compile("^/horse")}
            )
            for a in horse_a_list:
                horse_id = re.findall(r"\d+", a["href"])
                horse_id_list.append(horse_id[0])
            jockey_id_list = []
            jockey_a_list = soup.find("table", attrs={"summary": "レース結果"}).find_all(
                "a", attrs={"href": re.compile("^/jockey")}
            )
            for a in jockey_a_list:
                jockey_id = re.findall(r"\d+", a["href"])
                jockey_id_list.append(jockey_id[0])
            df["horse_id"] = horse_id_list
            df["jockey_id"] = jockey_id_list

            #インデックスをrace_idにする
            df.index = [race_id] * len(df)

            race_results[race_id] = df
          #存在しないrace_idを飛ばす
          except IndexError:
            continue
          #wifiの接続が切れた時などでも途中までのデータを返せるようにする
          except Exception as e:
            print(e)
            break
          #Jupyterで停止ボタンを押した時の対処
          except:
            break

        #pd.DataFrame型にして一つのデータにまとめる
        race_results_df = pd.concat([race_results[key] for key in race_results])

        return race_results_df

実際に2020年のデータを取得

- 年次
- 場所
  - 01: 札幌 02: 函館 03: 福島 04: 新潟 05: 東京06:中山07: 中京 08: 京都 09:阪神 10: 小倉
- レースの開催回数
- 開催日
- round

In [13]:
race_id_list = []
year = "2021"
for place in range(1,10,1):
  for kai in range(1, 13, 1):
    for day in range(1, 13, 1):
      for r in range(1, 13, 1):
          race_id = year + str(place).zfill(2) + str(kai).zfill(2) + str(day).zfill(2) + str(r).zfill(2)
          race_id_list.append(race_id)
    
len(race_id_list)

15552

In [ ]:
results = Results.scrape(race_id_list)

 データの保存

In [7]:
results.to_pickle('results_tmp.pickle')

## HorseResults.scrape()

馬の過去成績データをスクレイピングするメソッド



In [8]:
#馬の過去成績データを処理するクラス
class HorseResults:
  @staticmethod
  def scrape(horse_id_list):
      """
      馬の過去成績データをスクレイピングする関数

      Parameters:
      ----------
      horse_id_list : list
          馬IDのリスト

      Returns:
      ----------
      horse_results_df : pandas.DataFrame
          全馬の過去成績データをまとめてDataFrame型にしたもの
      """

      #horse_idをkeyにしてDataFrame型を格納
      horse_results = {}
      for horse_id in tqdm(horse_id_list):
          try:
              url = 'https://db.netkeiba.com/horse/' + horse_id
              df = pd.read_html(url)[3]
              #受賞歴がある馬の場合、3番目に受賞歴テーブルが来るため、4番目のデータを取得する
              if df.columns[0]=='受賞歴':
                  df = pd.read_html(url)[4]
              df.index = [horse_id] * len(df)
              horse_results[horse_id] = df
              time.sleep(1)
          except IndexError:
              continue
          except Exception as e:
              print(e)
              break
          except:
              break

      #pd.DataFrame型にして一つのデータにまとめる        
      horse_results_df = pd.concat([horse_results[key] for key in horse_results])

      return horse_results_df

上記で取得した馬idを元に馬の成績を保存

In [ ]:
horse_id_list = results['horse_id'].unique()
horse_results = HorseResults.scrape(horse_id_list)

馬成績の保存

In [ ]:
horse_results.to_pickle('horse_results_tmp.pickle')

## Peds.scrape()

馬ごとに3世代分の血統データをスクレイピングする

In [ ]:
#血統データを処理するクラス
class Peds:
    @staticmethod
    def scrape(horse_id_list):
        """
        血統データをスクレイピングする関数

        Parameters:
        ----------
        horse_id_list : list
            馬IDのリスト

        Returns:
        ----------
        peds_df : pandas.DataFrame
            全血統データをまとめてDataFrame型にしたもの
        """

        peds_dict = {}
        for horse_id in tqdm(horse_id_list):
            try:
            url = "https://db.netkeiba.com/horse/ped/" + horse_id
            df = pd.read_html(url)[0]

            #重複を削除して1列のSeries型データに直す
            generations = {}
            for i in reversed(range(3)):
                generations[i] = df[i]
                df.drop([i], axis=1, inplace=True)
                df = df.drop_duplicates()
            ped = pd.concat([generations[i] for i in range(5)]).rename(horse_id)

            peds_dict[horse_id] = ped.reset_index(drop=True)
            time.sleep(1)
            except IndexError:
            continue
            except Exception as e:
            print(e)
            break
            except:
            break

    #列名をpeds_0, ..., peds_61にする
    peds_df = pd.concat([peds_dict[key] for key in peds_dict], axis=1).T.add_prefix('peds_')

    return peds_df

血統データを取得

In [ ]:
horse_id_list = results['horse_id'].unique()
peds_results = Peds.scrape(horse_id_list)
peds_results #jupyterで出力

データの保存

In [ ]:
peds_results.to_pickle('peds_results_tmp.pickle')

## update_data関数

馬の過去成績データなどを新しくスクレイピングして古いデータに追加する

In [ ]:
def update_data(old, new):
    """
    Parameters:
    ----------
    old : pandas.DataFrame
        古いデータ
    new : pandas.DataFrame
        新しいデータ
    """

    filtered_old = old[~old.index.isin(new.index)]
    return pd.concat([filtered_old, new])

In [ ]:
update_data(results.pickle,results_tmp.pickle)
update_data(horse_results.pickle,horse_results_tmp.pickle)
update_data(peds_results.pickle,peds_results_tmp.pickle)